In [ ]:
# --- Cell 0: Load TIFF Series Data ---
%matplotlib widget

import importlib
import sys
from pathlib import Path

import ipywidgets as widgets
from ipyfilechooser import FileChooser
from IPython.display import display

import opym

# Reload modules to ensure they are up-to-date
try:
    importlib.reload(opym.dataloader)
    importlib.reload(opym.viewer)
    importlib.reload(opym)
    print("Reloaded opym.dataloader and opym.viewer")
except Exception as e:
    print(f"Note: Could not reload all modules: {e}")


# --- Create Interactive Widgets ---
print("\n1. Select any processed data directory (e.g., '.../DSR')")
print("2. Click 'Load Data'")

dir_chooser = FileChooser(
    path="/mmfs2/scratch/SDSMT.LOCAL/bscott/DataUpload",
    title="<b>Select Processed TIFF Directory to View:</b>",
    show_only_dirs=True,
)
load_button = widgets.Button(description="Load Data", button_style="primary")
view_output = widgets.Output()

# --- Global variable to hold viewer parameters ---
viewer_params = None


# --- Define the function to run on button click ---
def on_load_button_clicked(b):
    global viewer_params  # Make sure we're assigning to the global var

    with view_output:
        view_output.clear_output()

        if not dir_chooser.value:
            print(
                "❌ ERROR: No directory selected. Please select a folder and try again."
            )
            return

        data_dir = Path(dir_chooser.value)

        try:
            print(f"--- Loading data from: {data_dir.name} ---")

            # 1. Call the data loader
            # ---
            # FIX: Unpack all 9 return values from load_tiff_series
            # (get_stack, T_min, T_max, C_min, C_max, Z_max, Y, X, BASE_NAME)
            # ---
            (
                get_stack,
                t_min,
                t_max,
                c_min,
                c_max,
                z_max,
                y,
                x,
                base_name,
            ) = opym.load_tiff_series(data_dir)

            # 2. Store parameters in the global variable
            #    The viewers correctly need (get_stack, t_max, z_max, c_max, y, x)
            viewer_params = (get_stack, t_max, z_max, c_max, y, x)

            # ---
            # FIX: Update print statement to use the new t_min and c_min variables
            # ---
            print(
                f"✅ Data loaded. Shape: T={t_min}-{t_max}, "
                f"Z={z_max + 1}, C={c_min}-{c_max}, Y={y}, X={x}"
            )
            print(f"   Base name: {base_name}")
            print("\nYou can now run the viewer cells below.")

        except FileNotFoundError as e:
            print("\n❌ ERROR: Could not load data.")
            print(f"  Details: {e}")
            print(
                "  Please ensure you selected the correct directory containing the "
                "TIFF series."
            )
        except Exception as e:
            print(f"\n❌ An unexpected error occurred: {e}", file=sys.stderr)


# --- Link button and display UI ---
load_button.on_click(on_load_button_clicked)
display(widgets.VBox([dir_chooser, load_button, view_output]))

In [ ]:
# --- Cell 1: Launch the single-channel viewer ---
try:
    if viewer_params is None:
        raise NameError
    opym.single_channel_viewer(*viewer_params)
except NameError:
    print("❌ ERROR: 'viewer_params' not found. Please run Cell 0 to load data first.")
except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")

In [ ]:
# --- Cell 2: Launch the composite viewer ---
try:
    if viewer_params is None:
        raise NameError
    opym.composite_viewer(*viewer_params)
except NameError:
    print("❌ ERROR: 'viewer_params' not found. Please run Cell 0 to load data first.")
except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")